<a href="https://colab.research.google.com/github/InhyeokYoo/Pytorch-study/blob/master/Ch%207.%20%ED%95%99%EC%8A%B5%20%EC%8B%9C%20%EC%83%9D%EA%B8%B8%20%EC%88%98%20%EC%9E%88%EB%8A%94%20%EB%AC%B8%EC%A0%9C/%203.%20Dropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Settings

In [0]:
# Import libraries

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import numpy as np
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [0]:
# Hyper parameters

batch_size = 256
learning_rate = 0.0002
num_epochs = 10

# 2. Data

In [0]:
# Download data

mnist_train = dset.MNIST('./', train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = dset.MNIST('./', train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 20272111.05it/s]                            


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


32768it [00:00, 304966.51it/s]                           
0it [00:00, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw


1654784it [00:00, 5168948.15it/s]                           
8192it [00:00, 129221.01it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw
Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


In [0]:
# Set dataloader

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

# 3. Model & Optimizer

In [0]:
# 중간중간 dropout을 삽입하여 overfitting을 방지.

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,3,padding=1),  # 28
            nn.ReLU(),
            #nn.Dropout2d(0.2),
            nn.Conv2d(16,32,3,padding=1), # 28
            nn.ReLU(),
            #nn.Dropout2d(0.2),
            nn.MaxPool2d(2,2),            # 14
            nn.Conv2d(32,64,3,padding=1), # 14
            nn.ReLU(),
            #nn.Dropout2d(0.2),
            nn.MaxPool2d(2,2)             # 7
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64*7*7,100),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(100,10)
        )       
        
    def forward(self,x):
        out = self.layer(x)
        out = out.view(batch_size,-1)
        out = self.fc_layer(out)
        return out

In [0]:
# Loss & Optim

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

model = CNN().to(device)
loss_fnc = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

cuda:0


# 4. Train

In [0]:
for i in range(num_epochs):
    for j, (image, label) in enumerate(train_loader):
        x = image.to(device)
        y_ = label.to(device)

        optimizer.zero_grad()
        output = model(x)
        loss = loss_fnc(output, y_)

        loss.backward()
        optimizer.step()

    if i % 10 == 0:
        print(loss)

tensor(2.2996, device='cuda:0', grad_fn=<NllLossBackward>)


# 5. Test

In [0]:
correct = 0
total = 0

# batch normalization이나 drop out의 경우 test와 train시 다르게 작동하므로, eval을 해주어야 함.
model.eval()
with torch.no_grad():
    for image, label in test_loader:
        x = image.to(device)
        y_ = label.to(device)

        output = model(x)
        output_idx = torch.argmax(output, 1)

        total += label.size(0)
        correct += (output_idx == y_).sum().float()

    print("Accuracy of Test Data: {}".format(100*correct/total))

Accuracy of Test Data: 17.237581253051758
